## Loading packages and data

In [1]:
# Load packages
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
# Check working directory and paths
print("Working Directory: ", os.getcwd(), "\n", "Terminal Path: ", sys.path[0], sep='')

Working Directory: /Users/neelpendyala/Desktop
Terminal Path: /Users/neelpendyala/Desktop


In [3]:
# Set working directory to downloads folder
os.chdir("/Users/neelpendyala/Downloads")
print("New Working Directory: ", os.getcwd(), sep='')

New Working Directory: /Users/neelpendyala/Downloads


In [4]:
# Unzip 2020-21 play-by-play data
from zipfile import ZipFile
with ZipFile(os.getcwd() + '/NBA_PBP_2020-21.csv.zip', 'r') as f:
    f.extractall()

In [5]:
# Load 2020-21 play-by-play dataset
data = pd.read_csv('NBA_PBP_2020-21.csv', keep_default_na=False)

#play_in_game_dates = ['May 18 2021', 'May 19 2021', 'May 20 2021', 'May 21 2021']
#data = pd.read_csv('NBA_PBP_2020-21_full.csv', keep_default_na=False)
#data = data.query("GameType=='regular'")
#data = data.query("Date not in @play_in_game_dates")

working = data
#working.head()

***

## Part 1A

If ['ShotOutcome'] is blank, that means the play was a non-shot play, and instead involved some other event, e.g. offensive/defensive rebound, foul, turnover, etc.

## Part 1B

In [6]:
# Creating a new column with team 
working['WhichTeamMadePlay'] = np.where(working['HomePlay']!='', working['HomeTeam'], working['AwayTeam'])

In [7]:
# Finding highest assist combination per team
a = working.query("Assister!=''").groupby(['WhichTeamMadePlay', 'Assister', 'Shooter']).size().reset_index(name='Assists')
idx = a.groupby('WhichTeamMadePlay')['Assists'].idxmax()
b = a.loc[idx].reset_index(drop=True)

In [8]:
# Special characters in some players' names not read properly; player names before dash removed
assister = b['Assister'].map(lambda x: x.split(' - ', 1)[1])
shooter = b['Shooter'].map(lambda x: x.split(' - ', 1)[1])
combo = assister + " \u2794 " + shooter

In [9]:
# Final table showing which pair with most assist to made baskets for each team
each_team_highest_assist_combo = pd.DataFrame(
    {'Team': b['WhichTeamMadePlay'],
    'AssistMadeBasketCombo': combo,
    'Assists': b['Assists']}
)
each_team_highest_assist_combo

,Team,AssistMadeBasketCombo,Assists
0,ATL,youngtr01 ➔ collijo01,29
1,BOS,smartma01 ➔ brownja02,27
2,BRK,duranke01 ➔ harrijo01,18
3,CHI,whiteco01 ➔ cartewe01,19
4,CHO,ballla01 ➔ washipj01,20
5,CLE,osmande01 ➔ drumman01,14
6,DAL,doncilu01 ➔ hardati02,24
7,DEN,jokicni01 ➔ murraja01,29
8,DET,wrighde01 ➔ grantje01,21
9,GSW,greendr01 ➔ curryst01,20


## Part 1C

In [10]:
# First find top players in regular three point shot fouls
a = working.query("Quarter==4 & FreeThrowNum=='3 of 3'").groupby('FreeThrowShooter').size().reset_index(name='Fouls').sort_values(by='Fouls', ascending=False).reset_index(drop=True)

In [11]:
# Then, find top players in made three point shot fouls (and-ones)
working['2PTAndOnes'] = ''
working['3PTAndOnes'] = ''
working['AndOnes'] = ''
for i in range(working.shape[0]-1):
    if working.loc[i, 'ShotOutcome']=='make' and working.loc[i+1, 'FoulType']=='shooting' and working.loc[i, 'SecLeft']==working.loc[i+1, 'SecLeft'] and "2-pt" in working.loc[i, 'ShotType']:
        working.loc[i+1, '2PTAndOnes'] = working.loc[i+1, 'Fouled']
    elif working.loc[i, 'ShotOutcome']=='make' and working.loc[i+1, 'FoulType']=='shooting' and working.loc[i, 'SecLeft']==working.loc[i+1, 'SecLeft'] and "3-pt" in working.loc[i, 'ShotType']:
        working.loc[i+1, '3PTAndOnes'] = working.loc[i+1, 'Fouled']

    if working.loc[i+1, '2PTAndOnes']!='':
        working.loc[i+1, 'AndOnes'] = working.loc[i+1, '2PTAndOnes']
    elif working.loc[i+1, '3PTAndOnes']!='':
        working.loc[i+1, 'AndOnes'] = working.loc[i+1, '3PTAndOnes']

In [12]:
# Top players in three point and ones in fourth quarter
b = working.loc[working['3PTAndOnes']!=''].query("Quarter==4").groupby('3PTAndOnes').size().reset_index(name='Fouls').sort_values(by='Fouls', ascending=False).reset_index(drop=True)

In [13]:
# Cleaning up player names and renaming columns
player = a['FreeThrowShooter'].map(lambda x: x.split(' - ', 1)[1])
a['FreeThrowShooter'] = player
a = a.rename(columns={'FreeThrowShooter':'Player', 'Fouls':'3PTFouls'})

In [14]:
# Cleaning up player names and renaming columns
player = b['3PTAndOnes'].map(lambda x: x.split(' - ', 1)[1])
b['3PTAndOnes'] = player
b = b.rename(columns={'3PTAndOnes':'Player', 'Fouls':'3PTAndOnes'})

In [15]:
# Merge the data and generate final table showing players with most three point fouls drawn in fourth quarter
ab = a.merge(b, how='left', on='Player').replace(np.nan, 0).astype({'3PTAndOnes': 'int'})
ab['3PTTripstoLine'] = ab['3PTFouls'] + ab['3PTAndOnes']
ab = ab.sort_values(by='3PTTripstoLine', ascending=False).reset_index(drop=True)
top_10_in_4thQ_3ptfoulsdrawn = ab.head(10)
# Final table
top_10_in_4thQ_3ptfoulsdrawn

,Player,3PTFouls,3PTAndOnes,3PTTripstoLine
0,redicjj01,2,2,4
1,duranke01,2,1,3
2,gordoer01,2,0,2
3,robindu01,1,1,2
4,batumni01,1,1,2
5,muscami01,1,1,2
6,huntede01,1,0,1
7,mccolcj01,1,0,1
8,stanlca01,1,0,1
9,bertada01,1,0,1


In [16]:
# Generate final table showing players with most and-1s in the fourth quarter
c = working.loc[working['AndOnes']!=''].query("Quarter==4").groupby('AndOnes').size().reset_index(name='Fouls').sort_values(by='Fouls', ascending=False).reset_index(drop=True)
# Cleaning up player names and renaming columns
player = c['AndOnes'].map(lambda x: x.split(' - ', 1)[1])
c['AndOnes'] = player
c = c.rename(columns={'AndOnes':'Player', 'Fouls':'AndOnes'})
top_10_in_4thQ_AndOnes = c.head(10)
# Final table
top_10_in_4thQ_AndOnes

,Player,AndOnes
0,willizi01,6
1,harremo01,6
2,antetgi01,5
3,sabondo01,5
4,vandeja01,5
5,woodch01,4
6,jamesle01,4
7,derozde01,4
8,duranke01,3
9,johnsja01,3


***

## Part 2A

In [17]:
# Filter data to focus on team of interest (ATL Hawks)
part2_working = pd.concat([working.query("HomeTeam=='ATL'"), working.query("AwayTeam=='ATL'")], ignore_index=False).sort_index().reset_index()
# Checking number of total games (should be 72)
#part2_working['URL'].nunique()

#### EFG%

In [18]:
# Team EFG%, Wins
a = part2_working.query("WinningTeam=='ATL' & WhichTeamMadePlay=='ATL'").agg(ShotsMade = ('ShotOutcome', lambda x: sum(x=='make')),
                                                                         ShotsAttempted = ('ShotOutcome', lambda x: sum(x!=''))).transpose().reset_index(drop=True)
b = part2_working.query("WinningTeam=='ATL' & WhichTeamMadePlay=='ATL' & ShotType.str.contains('3-pt')").agg(ShotsMade = ('ShotOutcome', lambda x: sum(x=='make'))).transpose().reset_index(drop=True).rename(columns={'ShotsMade':'3PTShotsMade'})
efg_wins = pd.concat([a,b], axis=1).reindex(columns = ['ShotsMade', '3PTShotsMade', 'ShotsAttempted'])
efg_wins['EFG%'] = (efg_wins['ShotsMade'] + .5*efg_wins['3PTShotsMade'])/efg_wins['ShotsAttempted']
efg_wins.index = ['Wins']
#efg_wins

# Opp EFG%, Wins
a = part2_working.query("WinningTeam=='ATL' & WhichTeamMadePlay!='ATL'").agg(ShotsMade = ('ShotOutcome', lambda x: sum(x=='make')),
                                                                         ShotsAttempted = ('ShotOutcome', lambda x: sum(x!=''))).transpose().reset_index(drop=True)
b = part2_working.query("WinningTeam=='ATL' & WhichTeamMadePlay!='ATL' & ShotType.str.contains('3-pt')").agg(ShotsMade = ('ShotOutcome', lambda x: sum(x=='make'))).transpose().reset_index(drop=True).rename(columns={'ShotsMade':'3PTShotsMade'})
opp_efg_wins = pd.concat([a,b], axis=1).reindex(columns = ['ShotsMade', '3PTShotsMade', 'ShotsAttempted'])
opp_efg_wins['EFG%'] = (opp_efg_wins['ShotsMade'] + .5*opp_efg_wins['3PTShotsMade'])/opp_efg_wins['ShotsAttempted']
opp_efg_wins.index = ['Wins']
#opp_efg_wins

# Team EFG%, Losses
a = part2_working.query("WinningTeam!='ATL' & WhichTeamMadePlay=='ATL'").agg(ShotsMade = ('ShotOutcome', lambda x: sum(x=='make')),
                                                                         ShotsAttempted = ('ShotOutcome', lambda x: sum(x!=''))).transpose().reset_index(drop=True)
b = part2_working.query("WinningTeam!='ATL' & WhichTeamMadePlay=='ATL' & ShotType.str.contains('3-pt')").agg(ShotsMade = ('ShotOutcome', lambda x: sum(x=='make'))).transpose().reset_index(drop=True).rename(columns={'ShotsMade':'3PTShotsMade'})
efg_losses = pd.concat([a,b], axis=1).reindex(columns = ['ShotsMade', '3PTShotsMade', 'ShotsAttempted'])
efg_losses['EFG%'] = (efg_losses['ShotsMade'] + .5*efg_losses['3PTShotsMade'])/efg_losses['ShotsAttempted']
efg_losses.index = ['Losses']
#efg_losses

# Opp EFG%, Losses
a = part2_working.query("WinningTeam!='ATL' & WhichTeamMadePlay!='ATL'").agg(ShotsMade = ('ShotOutcome', lambda x: sum(x=='make')),
                                                                         ShotsAttempted = ('ShotOutcome', lambda x: sum(x!=''))).transpose().reset_index(drop=True)
b = part2_working.query("WinningTeam!='ATL' & WhichTeamMadePlay!='ATL' & ShotType.str.contains('3-pt')").agg(ShotsMade = ('ShotOutcome', lambda x: sum(x=='make'))).transpose().reset_index(drop=True).rename(columns={'ShotsMade':'3PTShotsMade'})
opp_efg_losses = pd.concat([a,b], axis=1).reindex(columns = ['ShotsMade', '3PTShotsMade', 'ShotsAttempted'])
opp_efg_losses['EFG%'] = (opp_efg_losses['ShotsMade'] + .5*opp_efg_losses['3PTShotsMade'])/opp_efg_losses['ShotsAttempted']
opp_efg_losses.index = ['Losses']
#opp_efg_losses

# EFG% Summary
efg_summary = pd.concat([pd.concat([efg_wins, opp_efg_wins], axis=1).iloc[:, [3,7]], pd.concat([efg_losses, opp_efg_losses], axis=1).iloc[:, [3,7]]],
                        axis=0)
efg_summary.columns.values[0] = "TeamEFG%"
efg_summary.columns.values[1] = "OppEFG%"
efg_summary['Difference'] = efg_summary['TeamEFG%'] - efg_summary['OppEFG%']
efg_summary = round(efg_summary*100, 2)
efg_summary

,TeamEFG%,OppEFG%,Difference
Wins,53.38,47.28,6.09
Losses,47.80,53.24,-5.44


#### FTA Rate

In [19]:
# Team FTA Rate, Wins
a = part2_working.query("WinningTeam=='ATL' & WhichTeamMadePlay=='ATL'").agg({'FreeThrowNum': lambda x: sum(x!=''),
                                                                              'ShotOutcome': lambda x: sum(x!='')})
ftarate_wins = pd.DataFrame(a).transpose().rename(columns={'FreeThrowNum':'FTAttempted', 'ShotOutcome':'ShotsAttempted'})
ftarate_wins['FTARate'] = ftarate_wins['FTAttempted']/ftarate_wins['ShotsAttempted']
ftarate_wins.index = ['Wins']
#ftarate_wins

# Opp FTA Rate, Wins
a = part2_working.query("WinningTeam=='ATL' & WhichTeamMadePlay!='ATL'").agg({'FreeThrowNum': lambda x: sum(x!=''),
                                                                              'ShotOutcome': lambda x: sum(x!='')})
opp_ftarate_wins = pd.DataFrame(a).transpose().rename(columns={'FreeThrowNum':'FTAttempted', 'ShotOutcome':'ShotsAttempted'})
opp_ftarate_wins['FTARate'] = opp_ftarate_wins['FTAttempted']/opp_ftarate_wins['ShotsAttempted']
opp_ftarate_wins.index = ['Wins']
opp_ftarate_wins

# Team FTA Rate, Losses
a = part2_working.query("WinningTeam!='ATL' & WhichTeamMadePlay=='ATL'").agg({'FreeThrowNum': lambda x: sum(x!=''),
                                                                              'ShotOutcome': lambda x: sum(x!='')})
ftarate_losses = pd.DataFrame(a).transpose().rename(columns={'FreeThrowNum':'FTAttempted', 'ShotOutcome':'ShotsAttempted'})
ftarate_losses['FTARate'] = ftarate_losses['FTAttempted']/ftarate_losses['ShotsAttempted']
ftarate_losses.index = ['Losses']
#ftarate_losses

# Opp FTA Rate, Losses
a = part2_working.query("WinningTeam!='ATL' & WhichTeamMadePlay!='ATL'").agg({'FreeThrowNum': lambda x: sum(x!=''),
                                                                              'ShotOutcome': lambda x: sum(x!='')})
opp_ftarate_losses = pd.DataFrame(a).transpose().rename(columns={'FreeThrowNum':'FTAttempted', 'ShotOutcome':'ShotsAttempted'})
opp_ftarate_losses['FTARate'] = opp_ftarate_losses['FTAttempted']/opp_ftarate_losses['ShotsAttempted']
opp_ftarate_losses.index = ['Losses']
#opp_ftarate_losses

# FTA Rate, Summary
ftarate_summary = pd.concat([pd.concat([ftarate_wins, opp_ftarate_wins], axis=1).iloc[:, [2,5]], pd.concat([ftarate_losses, opp_ftarate_losses], axis=1).iloc[:, [2,5]]],
                        axis=0)
ftarate_summary.columns.values[0] = "TeamFTARate"
ftarate_summary.columns.values[1] = "OppFTARate"
ftarate_summary['Difference'] = ftarate_summary['TeamFTARate'] - ftarate_summary['OppFTARate']
ftarate_summary = round(ftarate_summary, 4)
ftarate_summary

,TeamFTARate,OppFTARate,Difference
Wins,0.3264,0.2360,0.0903
Losses,0.2614,0.2464,0.0150


Calculating new columns to help in the calculation of TOV% and OREB%:

In [20]:
# Performing operations to calculate TOV% and OREB%
part2_working['ReboundType_2'] = part2_working['ReboundType']
part2_working[['FreeThrowNumCurrent', 'FreeThrowNumAttempts']] = part2_working['FreeThrowNum'].str.split(' of ', expand=True)
part2_working['FreeThrowNumAttempts'] = part2_working['FreeThrowNumAttempts'].apply(lambda x: x if x is not None else '')    

In [21]:
# Operations continued -- creating column that has whether the free throw was the last one or not
part2_working['isLastFreeThrow'] = ''
for i in range(part2_working.shape[0]):
    if (part2_working.loc[i, 'FreeThrowNumCurrent']!='') & (part2_working.loc[i, 'FreeThrowNumAttempts']!='') & (part2_working.loc[i, 'FreeThrowNumCurrent']== part2_working.loc[i, 'FreeThrowNumAttempts']):
        part2_working.loc[i, 'isLastFreeThrow'] = 'last free throw'

In [22]:
# Create a new rebound column that does not include rebounds after technical fouls and rebounds after non-possession ending free throws
for i in range(part2_working.shape[0]-1):
    if (part2_working.loc[i, 'isLastFreeThrow']=='last free throw' and part2_working.loc[i, 'FreeThrowOutcome']=='miss') or (part2_working.loc[i, 'ShotOutcome']=='miss'):
        part2_working.loc[i+1, 'ReboundType_2'] = part2_working.loc[i+1, 'ReboundType']
    else:
        part2_working.loc[i+1, 'ReboundType_2'] = ''

In [23]:
# Manual rebound corrections based on vieweing the data outside of Jupyter Notebook
#part2_working.loc[1776, 'ReboundType_2'] = part2_working.loc[1776, 'ReboundType']
#part2_working.loc[22379, 'ReboundType_2'] = ''
#part2_working.loc[22380, 'ReboundType_2'] = part2_working.loc[22380, 'ReboundType']
#part2_working.loc[25295, 'ReboundType_2'] = part2_working.loc[25295, 'ReboundType']
#part2_working.loc[28173, 'ReboundType_2'] = ''
#part2_working.loc[28174, 'ReboundType_2'] = part2_working.loc[28174, 'ReboundType']

In [24]:
# Based on online research, team rebounds are not assigned to an individual player as they were not capable of being collected by an individual player
# and therefore will be treated as a rebound that was not available
# Creating another column that ignores team rebounds
part2_working['ReboundType_3'] = np.where(part2_working['Rebounder']=='Team', '', part2_working['ReboundType_2'])

Was unable to calculate TOV% and OREB% and get the resulting values to match the official NBA Stats website. Therefore, TOV/GM and OREB/GM were calculated instead. 

#### TOV%

In [25]:
# Team TOV/GM, Wins
a = part2_working.query("WinningTeam=='ATL' & WhichTeamMadePlay=='ATL'").agg({'URL': lambda x: x.nunique(),
                                                                              'TurnoverPlayer': lambda x: sum(x!='')})
tov_wins = pd.DataFrame(a).transpose().rename(columns={'URL':'Games', 'TurnoverPlayer':'Turnovers'})
tov_wins['TOV/GM'] = tov_wins['Turnovers']/tov_wins['Games'] 
tov_wins.index = ['Wins']
#tov_wins

# Opp TOV/GM, Wins
a = part2_working.query("WinningTeam=='ATL' & WhichTeamMadePlay!='ATL'").agg({'URL': lambda x: x.nunique(),
                                                                              'TurnoverPlayer': lambda x: sum(x!='')})
opp_tov_wins = pd.DataFrame(a).transpose().rename(columns={'URL':'Games', 'TurnoverPlayer':'Turnovers'})
opp_tov_wins['TOV/GM'] = opp_tov_wins['Turnovers']/opp_tov_wins['Games'] 
opp_tov_wins.index = ['Wins']
#opp_tov_wins

# Team TOV/GM, Losses
a = part2_working.query("WinningTeam!='ATL' & WhichTeamMadePlay=='ATL'").agg({'URL': lambda x: x.nunique(),
                                                                              'TurnoverPlayer': lambda x: sum(x!='')})
tov_losses = pd.DataFrame(a).transpose().rename(columns={'URL':'Games', 'TurnoverPlayer':'Turnovers'})
tov_losses['TOV/GM'] = tov_losses['Turnovers']/tov_losses['Games'] 
tov_losses.index = ['Losses']
#tov_losses

# Opp TOV/GM, Losses
a = part2_working.query("WinningTeam!='ATL' & WhichTeamMadePlay!='ATL'").agg({'URL': lambda x: x.nunique(),
                                                                              'TurnoverPlayer': lambda x: sum(x!='')})
opp_tov_losses = pd.DataFrame(a).transpose().rename(columns={'URL':'Games', 'TurnoverPlayer':'Turnovers'})
opp_tov_losses['TOV/GM'] = opp_tov_losses['Turnovers']/opp_tov_losses['Games'] 
opp_tov_losses.index = ['Losses']
#tov_losses

# TOV/GM, Summary
tov_summary = pd.concat([pd.concat([tov_wins, opp_tov_wins], axis=1).iloc[:, [2,5]], pd.concat([tov_losses, opp_tov_losses], axis=1).iloc[:, [2,5]]],
                        axis=0)
tov_summary.columns.values[0] = "TeamTOV/GM"
tov_summary.columns.values[1] = "OppTOV/GM"
tov_summary['Difference'] = tov_summary['OppTOV/GM'] - tov_summary['TeamTOV/GM']
tov_summary = round(tov_summary, 2)
tov_summary

,TeamTOV/GM,OppTOV/GM,Difference
Wins,15.71,12.86,-2.86
Losses,14.43,13.00,-1.43


#### OREB%

In [26]:
# Team OREB/GM, Wins
a = part2_working.query("WinningTeam=='ATL' & WhichTeamMadePlay=='ATL'").agg({'URL': lambda x: x.nunique(),
                                                                              'ReboundType_3': lambda x: sum(x=='offensive')})
oreb_wins = pd.DataFrame(a).transpose().rename(columns={'URL':'Games', 'ReboundType_3':'Turnovers'})
oreb_wins['OREB/GM'] = oreb_wins['Turnovers']/oreb_wins['Games'] 
oreb_wins.index = ['Wins']
#oreb_wins

# Opp OREB/GM, Wins
a = part2_working.query("WinningTeam=='ATL' & WhichTeamMadePlay!='ATL'").agg({'URL': lambda x: x.nunique(),
                                                                              'ReboundType_3': lambda x: sum(x=='offensive')})
opp_oreb_wins = pd.DataFrame(a).transpose().rename(columns={'URL':'Games', 'ReboundType_3':'Turnovers'})
opp_oreb_wins['OREB/GM'] = opp_oreb_wins['Turnovers']/opp_oreb_wins['Games'] 
opp_oreb_wins.index = ['Wins']
#opp_oreb_wins

# Team OREB/GM, Losses
a = part2_working.query("WinningTeam!='ATL' & WhichTeamMadePlay=='ATL'").agg({'URL': lambda x: x.nunique(),
                                                                              'ReboundType_3': lambda x: sum(x=='offensive')})
oreb_losses = pd.DataFrame(a).transpose().rename(columns={'URL':'Games', 'ReboundType_3':'Turnovers'})
oreb_losses['OREB/GM'] = oreb_losses['Turnovers']/oreb_losses['Games'] 
oreb_losses.index = ['Losses']
#oreb_losses

# Opp OREB/GM, Losses
a = part2_working.query("WinningTeam!='ATL' & WhichTeamMadePlay!='ATL'").agg({'URL': lambda x: x.nunique(),
                                                                              'ReboundType_3': lambda x: sum(x=='offensive')})
opp_oreb_losses = pd.DataFrame(a).transpose().rename(columns={'URL':'Games', 'ReboundType_3':'Turnovers'})
opp_oreb_losses['OREB/GM'] = opp_oreb_losses['Turnovers']/opp_oreb_losses['Games'] 
opp_oreb_losses.index = ['Losses']
#oreb_losses

# OREB/GM, Summary
oreb_summary = pd.concat([pd.concat([oreb_wins, opp_oreb_wins], axis=1).iloc[:, [2,5]], pd.concat([oreb_losses, opp_oreb_losses], axis=1).iloc[:, [2,5]]],
                        axis=0)
oreb_summary.columns.values[0] = "TeamOREB/GM"
oreb_summary.columns.values[1] = "OppOREB/GM"
oreb_summary['Difference'] = oreb_summary['TeamOREB/GM'] - oreb_summary['OppOREB/GM']
oreb_summary = round(oreb_summary, 2)
oreb_summary

,TeamOREB/GM,OppOREB/GM,Difference
Wins,11.86,9.86,2.00
Losses,12.29,11.00,1.29


The four factors (EFG%, FTA Rate, TOV/GM, OREB/GM) have been calculated above. EFG% and FTA Rate are the most important four factors as seen in the difference of the values between wins and losses. TOV% and OREB% are not as important because the extra few possessions/scoring opportunities that are gained as a result of TOVs and OREBs do not signficantly impact the final score/outcome of the game. 

***

## References

[1] https://stackoverflow.com/<br>
[2] https://saturncloud.io/<br>
[3] https://www.geeksforgeeks.org<br>
[4] https://www.w3schools.com/<br>
[5] https://pandas.pydata.org/<br>